# XAUUSD LSTM Colab AutoLoad
مدل LSTM برای پیش‌بینی قیمت طلا (XAUUSD)

In [ ]:
# این سلول برای نصب کتابخانه‌های لازم است
!pip install numpy pandas scikit-learn tensorflow matplotlib

In [ ]:
# این سلول برای بارگذاری فایل CSV از GitHub است
import pandas as pd
url = 'https://raw.githubusercontent.com/behicof/new-lsdm/main/xauusd_lstm_ready_clean.csv'
df = pd.read_csv(url)
df.head()

In [ ]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# انتخاب ویژگی‌ها
features = ['close', 'rsi', 'atr', 'ema_diff', 'body', 'upper_shadow', 'lower_shadow']
data = df[features].dropna()

# مقیاس‌بندی داده‌ها
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# ساخت توالی‌ها
X, y = [], []
sequence_length = 60
for i in range(sequence_length, len(scaled_data) - 10):
    X.append(scaled_data[i-sequence_length:i])
    now = data['close'].iloc[i]
    future = data['close'].iloc[i + 10]
    delta = future - now
    threshold = data['atr'].iloc[i] * 1.5
    if delta > threshold:
        y.append(2)  # سیگنال خرید
    elif delta < -threshold:
        y.append(0)  # سیگنال فروش
    else:
        y.append(1)  # نگه‌داری

X, y = np.array(X), np.array(y)

# تقسیم داده به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ساخت مدل LSTM
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# آموزش مدل
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# ارزیابی مدل
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
